This notebook shows how to use `AdversarialAttackEvaluator` to evaluate the performence of adversarial attacks.

In [ ]:
# import libraries
import torch
import matplotlib.pyplot as plt
import advsecurenet.shared.types.configs.attack_configs as AttackConfigs

from advsecurenet.models.model_factory import ModelFactory
from advsecurenet.datasets import DatasetFactory
from advsecurenet.dataloader import DataLoaderFactory
from advsecurenet.attacks import PGD, CWAttack, FGSM, LOTS, DeepFool
from advsecurenet.shared.types import DatasetType, ModelType
from advsecurenet.utils.model_utils import load_model, download_weights
from advsecurenet.utils.evaluation import AdversarialAttackEvaluator


In [ ]:
# first download the weights
download_weights(filename="CustomMnistModel_mnist_weights.pth")

In [ ]:
# get the model
model = ModelFactory.create_model('CustomMnistModel', num_classes=10, num_channels=1) # this assumes that the CustomMnistModel is defined in the models/CustomMnistModel.py file
model = load_model(model, 'CustomMnistModel_mnist_weights.pth')


In [ ]:
# get the dataset
dataset_obj =DatasetFactory.create_dataset(DatasetType.MNIST)
#trainData = datasetObj.load_dataset(train=True)
test_data = dataset_obj.load_dataset(train=False)

In [ ]:
test_data_loader = DataLoaderFactory.create_dataloader(test_data,batch_size=32, shuffle=False)


In [ ]:
evaluator = AdversarialAttackEvaluator()

In [ ]:
fgsm_config = AttackConfigs.FgsmAttackConfig(
    epsilon=0.05
)
fgsm = FGSM(fgsm_config)

In [ ]:
# save the results
experiment_info = {
    "model_name": "CustomMnistModel",
    "dataset_name": "MNIST",
    "attack_name": "FGSM",
    "is_targeted": False,
    "researcher": "Melih Catal"
} 
file_name = "results.csv"
for images, labels in test_data_loader:
    adv_img = fgsm.attack(model, images, labels)
    results = evaluator.full_evaluation(model, original_images=images, true_labels=labels, adversarial_images=adv_img, is_targeted=False)
    evaluator.save_results_to_csv(results, experiment_info, file_name=file_name)
    

In [ ]:
evaluator.evaluate_attack(model, images, labels, adv_img)

In [ ]:
evaluator.calculate_perturbation_distances(images, adv_img)

In [ ]:
evaluator.calculate_robustness_gap(model, images, labels, adv_img)

In [ ]:
evaluator.calculate_ssim(images, adv_img)

In [ ]:
evaluator.calculate_similarity_scores(images, adv_img)

In [ ]:
target_model = ModelFactory.create_model("CustomMnistModel", num_classes=10, num_channels=1)

In [ ]:
evaluator.calculate_transferability_rate(source_model=model, target_model=target_model, original_images=images, true_labels=labels, adversarial_images=adv_img, is_targeted=False)

In [ ]:
results = evaluator.full_evaluation(model, original_images=images, true_labels=labels, adversarial_images=adv_img, is_targeted=False)
results

In [ ]:
# save the results
experiment_info = {
    "model_name": "CustomMnistModel",
    "dataset_name": "MNIST",
    "attack_name": "FGSM",
    "is_targeted": False,
    "researcher": "Melih Catal"
} 
evaluator.save_results_to_csv(results, experiment_info)